In [35]:
import pandas as pd
import numpy as np

In [49]:
# read the file from github and create a dataframe
url = 'https://raw.githubusercontent.com/robertochiaiese/datawarehouse/refs/heads/main/dataset/ecdc-data/cases_deaths.csv'
df = pd.read_csv(url)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129270 entries, 0 to 129269
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   country       129270 non-null  object 
 1   country_code  125756 non-null  object 
 2   continent     129270 non-null  object 
 3   population    129270 non-null  int64  
 4   indicator     129270 non-null  object 
 5   daily_count   129270 non-null  int64  
 6   date          129270 non-null  object 
 7   rate_14_day   123628 non-null  float64
 8   source        129270 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 8.9+ MB


In [51]:
# upload dimCountry and dimDate to create the fact table for the cases and the deaths
dimCountry = pd.read_csv('/home/jovyan/datawarehouse/core_layer/to_get_ready/population_by_age.csv')
dimDate = pd.read_csv('/home/jovyan/datawarehouse/lookup/dim_date.csv')

In [60]:
#merge both dimCountry and dimDate
factCasesAndDeaths = pd.merge(df,dimCountry, left_on='country', right_on='country')
factCasesAndDeaths = pd.merge(factCasesAndDeaths,dimDate, left_on='date', right_on='date')

In [64]:
# filter the fact table for the cases and the deaths selecting only EU countries and exclude the rows whose country_id are empty 
factCasesAndDeaths = factCasesAndDeaths[(factCasesAndDeaths['country_code_2_digit'].notnull()) & (factCasesAndDeaths['continent']=='Europe')]

In [67]:
factCasesAndDeaths.head(1)

,country,country_code,continent,population_x,indicator,daily_count,date,rate_14_day,source,Unnamed: 0,...,year,month,day,day_name,day_of_year,week_of_month,week_of_year,month_name,year_month,year_week
0,Albania,ALB,Europe,2862427,confirmed cases,0,2020-01-02,NaN,"Epidemic intelligence, national daily data",0,...,2020,1,2,Thursday,2,1,1,January,202001,202001


In [73]:
# rename the columns
factCasesAndDeaths = factCasesAndDeaths[['country_code_2_digit','date_key','daily_count','source']].rename(columns={'country_code_2_digit':'country_id','date_key':'date_id','daily_cound':'confirmed_cases'})


In [78]:
# create an index
factCasesAndDeaths = factCasesAndDeaths.reset_index().rename(columns={'index':'cases_deaths_id'})


In [79]:
factCasesAndDeaths

,cases_deaths_id,country_id,date_id,confirmed_cases,source
0,0,AL,20200102,0,"Epidemic intelligence, national daily data"
1,1,AL,20200102,0,"Epidemic intelligence, national daily data"
2,2,AD,20200102,0,"Epidemic intelligence, national daily data"
3,3,AD,20200102,0,"Epidemic intelligence, national daily data"
4,4,AM,20200102,0,"Epidemic intelligence, national daily data"
...,...,...,...,...,...
27411,27411,TR,20201025,69,"Epidemic intelligence, national daily data"
27412,27412,UA,20201025,7014,"Epidemic intelligence, national daily data"
27413,27413,UA,20201025,125,"Epidemic intelligence, national daily data"
27414,27414,UK,20201025,23012,"Epidemic intelligence, national daily data"


In [ ]:
factCasesAndDeathsto_csv('/home/jovyan/datawarehouse/core_layer/processed/factCasesAndDeaths.csv', index=False)